In [11]:
dataset_id = "fda_staging"

In [45]:
!bq --location=US mk --dataset {dataset_id}

BigQuery error in mk operation: Dataset 'still-primer-302701:fda_staging'
already exists.


In [13]:
!bq --location=US load --autodetect --skip_leading_rows=1 --allow_jagged_rows=true --source_format=CSV {dataset_id}.US_COVID_Cases \
gs://covid_milestone/us_covid/*.csv 

Waiting on bqjob_r13c8987ede88be60_00000178e300bfe5_1 ... (1s) Current status: DONE   


In [2]:
dataset_id = "uscb_staging"

In [3]:
bucket_name = "covid_milestone"

In [4]:
!bq --location=US mk --dataset {dataset_id}

BigQuery error in mk operation: Dataset 'still-primer-302701:uscb_staging'
already exists.


In [9]:
!bq --location=US load --autodetect --skip_leading_rows=2 --source_format=CSV {dataset_id}.Social_Characteristics \
gs://{bucket_name}/state_char/*.csv

Waiting on bqjob_r5471224cd7e4b7ea_00000178e2fa88ae_1 ... (4s) Current status: DONE   


In [14]:
%%bigquery
select count(*) as count from fda_staging.US_COVID_Cases

,count
0,21478


In [10]:
%%bigquery
select count(*) as count from uscb_staging.Social_Characteristics

,count
0,50


## Get sample records

In [64]:
%%bigquery
select * from fda_staging.US_COVID_Cases
limit 5

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,UID,ISO3,Testing_Rate,Hospitalization_Rate
0,Northern Mariana Islands,US,2020-07-19 04:35:12+00:00,15.0979,145.6739,37,2,29.0,6.0,69.0,67.097055,10701.0,4,5.405405,580.0,MNP,19405.556362,10.81081081081081
1,Northern Mariana Islands,US,2020-08-27 04:30:22+00:00,15.0979,145.6739,54,2,29.0,23.0,69.0,97.925432,12921.0,4.0,3.703704,580.0,MNP,23431.379661,7.407407407407407
2,Northern Mariana Islands,US,2020-07-17 04:35:02+00:00,15.0979,145.6739,37,2,29.0,6.0,69.0,67.097055,10700.0,4,5.405405,580.0,MNP,19403.742928,10.81081081081081
3,Northern Mariana Islands,US,2020-08-21 04:30:30+00:00,15.0979,145.6739,54,2,29.0,23.0,69.0,97.925432,12921.0,4.0,3.703704,580.0,MNP,23431.379661,7.407407407407407
4,Northern Mariana Islands,US,2020-08-28 04:30:22+00:00,15.0979,145.6739,54,2,29.0,23.0,69.0,97.925432,13540.0,4.0,3.703704,580.0,MNP,24553.895256,7.407407407407407


In [17]:
%%bigquery
select * from uscb_staging.Social_Characteristics
limit 5

,id,Geographic_Area_Name,Estimate__HOUSEHOLDS_BY_TYPE__Total_households,Margin_of_Error__HOUSEHOLDS_BY_TYPE__Total_households,Percent__HOUSEHOLDS_BY_TYPE__Total_households,Percent_Margin_of_Error__HOUSEHOLDS_BY_TYPE__Total_households,Estimate__HOUSEHOLDS_BY_TYPE__Total_households__Married_couple_family,Margin_of_Error__HOUSEHOLDS_BY_TYPE__Total_households__Married_couple_family,Percent__HOUSEHOLDS_BY_TYPE__Total_households__Married_couple_family,Percent_Margin_of_Error__HOUSEHOLDS_BY_TYPE__Total_households__Married_couple_family,...,Percent__COMPUTERS_AND_INTERNET_USE__Total_households,Percent_Margin_of_Error__COMPUTERS_AND_INTERNET_USE__Total_households,Estimate__COMPUTERS_AND_INTERNET_USE__Total_households__With_a_computer,Margin_of_Error__COMPUTERS_AND_INTERNET_USE__Total_households__With_a_computer,Percent__COMPUTERS_AND_INTERNET_USE__Total_households__With_a_computer,Percent_Margin_of_Error__COMPUTERS_AND_INTERNET_USE__Total_households__With_a_computer,Estimate__COMPUTERS_AND_INTERNET_USE__Total_households__With_a_broadband_Internet_subscription,Margin_of_Error__COMPUTERS_AND_INTERNET_USE__Total_households__With_a_broadband_Internet_subscription,Percent__COMPUTERS_AND_INTERNET_USE__Total_households__With_a_broadband_Internet_subscription,Percent_Margin_of_Error__COMPUTERS_AND_INTERNET_USE__Total_households__With_a_broadband_Internet_subscription
0,0400000US21,Kentucky,1748732,8789,1748732,(X),830268,11913,47.5,0.6,...,1748732,(X),1576667,10430,90.2,0.4,1454017,12385,83.1,0.5
1,0400000US01,Alabama,1897576,10370,1897576,(X),892050,13184,47.0,0.7,...,1897576,(X),1696826,11925,89.4,0.4,1548299,13378,81.6,0.5
2,0400000US41,Oregon,1649352,8550,1649352,(X),780807,11294,47.3,0.6,...,1649352,(X),1566512,9925,95.0,0.3,1468212,11348,89.0,0.4
3,0400000US09,Connecticut,1377166,6712,1377166,(X),649536,9097,47.2,0.6,...,1377166,(X),1282417,7322,93.1,0.4,1219426,8086,88.5,0.5
4,0400000US20,Kansas,1138329,6595,1138329,(X),571620,6915,50.2,0.6,...,1138329,(X),1057634,8021,92.9,0.4,969907,9411,85.2,0.7


# Join two datasets to see if there is a relationship between COVID and household types/ratios across each state

## Query finding out what states that have more than 1000000 households have top 5 (high) covid confirmed cases

In [65]:
%%bigquery
select u.Province_State as state, sum(u.Confirmed) as confirmed from uscb_staging.Social_Characteristics s 
join fda_staging.US_COVID_Cases u on u.Province_State = s.Geographic_Area_Name
where (s.Estimate__HOUSEHOLDS_BY_TYPE__Total_households) > 1000000
group by u.Province_State
order by sum(u.confirmed) DESC
limit 10

,state,confirmed
0,California,535221539
1,Texas,426849072
2,Florida,332225768
3,New York,290049972
4,Illinois,204931513
5,Georgia,165553411
6,Ohio,143577392
7,Pennsylvania,142972034
8,New Jersey,140395584
9,North Carolina,131931999


## Query finding out what states that have more than 1,000,000 housholds have the most recovered patients from COVID

In [69]:
%%bigquery
select u.Province_State as state, sum(u.Recovered) as recovered from fda_staging.US_COVID_Cases u
join uscb_staging.Social_Characteristics s on u.Province_State = s.Geographic_Area_Name
where (s.Estimate__HOUSEHOLDS_BY_TYPE__Total_households) > 1000000
group by u.Province_State
order by sum(u.recovered) DESC
limit 5

,state,recovered
0,Texas,252311763.0
1,Ohio,81935840.0
2,Tennessee,80889238.0
3,Pennsylvania,75314534.0
4,North Carolina,70445365.0


## Query finding out what states that have more than 1,000,000 housholds have the most deaths related to covid

In [71]:
%%bigquery
select u.Province_State as state, sum(u.Deaths) as deaths from fda_staging.US_COVID_Cases u
join uscb_staging.Social_Characteristics s on u.Province_State = s.Geographic_Area_Name
where (s.Estimate__HOUSEHOLDS_BY_TYPE__Total_households) > 1000000
group by u.Province_State
order by sum(u.deaths) DESC
limit 5

,state,deaths
0,New York,13210416
1,California,8114268
2,Texas,7341399
3,New Jersey,6203255
4,Florida,5726722


## Query finding out what states that have more than 30% percent married couple rate in total households have the most deaths related to covid


In [67]:
%%bigquery
select u.Province_State as state, sum(u.Confirmed) as confirmed from uscb_staging.Social_Characteristics s 
join fda_staging.US_COVID_Cases u on u.Province_State = s.Geographic_Area_Name
where (s.Percent__HOUSEHOLDS_BY_TYPE__Total_households__Married_couple_family) > 30
group by u.Province_State
order by sum(u.confirmed) DESC
limit 10

,state,confirmed
0,California,535221539
1,Texas,426849072
2,Florida,332225768
3,New York,290049972
4,Illinois,204931513
5,Georgia,165553411
6,Ohio,143577392
7,Pennsylvania,142972034
8,New Jersey,140395584
9,North Carolina,131931999


## Query finding out what states that have more than 30% percent married couple rate in total households have the most recoverd patients from covid

In [68]:
%%bigquery
select u.Province_State as state, sum(u.Recovered) as confirmed from uscb_staging.Social_Characteristics s 
join fda_staging.US_COVID_Cases u on u.Province_State = s.Geographic_Area_Name
where (s.Percent__HOUSEHOLDS_BY_TYPE__Total_households__Married_couple_family) > 30
group by u.Province_State
order by sum(u.recovered) DESC
limit 10

,state,confirmed
0,Texas,252311763.0
1,Ohio,81935840.0
2,Tennessee,80889238.0
3,Pennsylvania,75314534.0
4,North Carolina,70445365.0
5,Wisconsin,62898072.0
6,Minnesota,55052487.0
7,Michigan,54250330.0
8,Louisiana,51046120.0
9,Massachusetts,48605219.0


# After looking at the national data we decided to also use the US COVID Data and looked at trends in that dataset which pertained to the different states.

## Look at the number of COVID cases, COVID related Deaths and number of recovered in the five states that had the smallest percent population that was born in the US

In [46]:
%%bigquery
select u.Province_State, sum(u.confirmed) as total_confirmed, sum(u.deaths) as total_deaths, sum(u.recovered) as total_recovered
from fda_staging.US_COVID_Cases u
join uscb_staging.Social_Characteristics sc on u.Province_State = sc.Geographic_Area_Name
group by u.Province_State
order by sum(sc.Percent__PLACE_OF_BIRTH__Total_population__Native__Born_in_United_States__State_of_residence) ASC
limit 5

,Province_State,total_confirmed,total_deaths,total_recovered
0,Nevada,48414578,807913,275924.0
1,Florida,332225768,5726722,0.0
2,District of Columbia,7797375,249150,3935693.0
3,Arizona,129040004,2591166,12439312.0
4,New Hampshire,10351881,220000,5972572.0


## Look at the number of COVID cases, COVID related Deaths and number of recovered in the five states that had the highest percent population that was born in the US

In [59]:
%%bigquery
select u.Province_State, sum(u.confirmed) as total_confirmed, sum(u.deaths) as total_deaths, sum(u.recovered) as total_recovered
from fda_staging.US_COVID_Cases u
join uscb_staging.Social_Characteristics sc on u.Province_State = sc.Geographic_Area_Name
group by u.Province_State
order by sum(sc.Percent__PLACE_OF_BIRTH__Total_population__Native__Born_in_United_States__State_of_residence) DESC
limit 5

,Province_State,total_confirmed,total_deaths,total_recovered
0,Puerto Rico,18362485,339514,8692236.0
1,Louisiana,78428770,2161058,51046120.0
2,Michigan,111585274,3528701,54250330.0
3,Ohio,143577392,3007508,81935840.0
4,Mississippi,50432920,1247648,31269115.0


## Look at the number of COVID cases, COVID related Deaths and number of recovered in the five states that had the highest percent population  with a Bachelors degree or higher

In [58]:
%%bigquery
select u.Province_State, sum(u.confirmed) as total_confirmed, sum(u.deaths) as total_deaths, sum(u.recovered) as total_recovered
from fda_staging.US_COVID_Cases u
join uscb_staging.Social_Characteristics sc on u.Province_State = sc.Geographic_Area_Name
group by u.Province_State
order by sum(sc.Percent__EDUCATIONAL_ATTAINMENT__Population_25_years_and_over__Bachelor_s_degree) DESC
limit 5

,Province_State,total_confirmed,total_deaths,total_recovered
0,Colorado,67714262,1149900,2844365.0
1,District of Columbia,7797375,249150,3935693.0
2,New Jersey,140395584,6203255,11916136.0
3,Massachusetts,96289258,3864806,48605219.0
4,Minnesota,80920383,1216137,55052487.0


## Look at the number of COVID cases, COVID related Deaths and number of recovered in the five states that had the highest percent population  with a 9th degree diploma or less

In [61]:
%%bigquery
select u.Province_State, sum(u.confirmed) as total_confirmed, sum(u.deaths) as total_deaths, sum(u.recovered) as total_recovered
from fda_staging.US_COVID_Cases u
join uscb_staging.Social_Characteristics sc on u.Province_State = sc.Geographic_Area_Name
group by u.Province_State
order by sum(sc.Percent__EDUCATIONAL_ATTAINMENT__Population_25_years_and_over__Less_than_9th_grade) DESC
limit 5

,Province_State,total_confirmed,total_deaths,total_recovered
0,Puerto Rico,18362485,339514,8692236.0
1,California,535221539,8114268,0.0
2,Texas,426849072,7341399,252311763.0
3,New Mexico,28556043,588468,11019177.0
4,New York,290049972,13210416,25612871.0


## Look at the number of COVID cases, COVID related Deaths and number of recovered in the five states that had the highest percent population with English Ancestry

In [63]:
%%bigquery
select u.Province_State, sum(u.confirmed) as total_confirmed, sum(u.deaths) as total_deaths, sum(u.recovered) as total_recovered
from fda_staging.US_COVID_Cases u
join uscb_staging.Social_Characteristics sc on u.Province_State = sc.Geographic_Area_Name
group by u.Province_State
order by sum(sc.Percent__ANCESTRY__Total_population__English) DESC
limit 5

,Province_State,total_confirmed,total_deaths,total_recovered
0,Utah,57882522,304660,34250078.0
1,Maine,6066302,103009,1940905.0
2,Vermont,2104402,36831,989871.0
3,New Hampshire,10351881,220000,5972572.0
4,Idaho,28375228,299542,9744844.0
